In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import Dropout,LSTM,Dense
from keras.layers import LeakyReLU


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import hilbert_data1_jgrjd_20CRV3


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier, plot_tree, XGBRegressor
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
#import hilbert_data 

In [4]:
!pwd

/home/cccr/supriyo/panini/filtered_data/historical/JGRJD/stacked_lstm_butter_filter


### Test and Train Splitter:

In [7]:
x_train,_,y_train = hilbert_data1_jgrjd_20CRV3.data_hilbert(datetime.datetime(1979,1,1),datetime.datetime(2008,12,31))
x_test,_,y_test = hilbert_data1_jgrjd_20CRV3.data_hilbert(datetime.datetime(1974,6,1),datetime.datetime(1978,3,16))

FileNotFoundError: [Errno 2] File b'full_data_nn_20CR_V3.txt' does not exist: b'full_data_nn_20CR_V3.txt'

In [ ]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
#y_train.head()

In [ ]:
abs(x_train.describe()) - abs(x_test.describe())

In [ ]:
x_test.describe()

In [ ]:
x_train['doy'] = x_train.index.month
x_test['doy']  = x_test.index.month

In [ ]:
#x_train['max'] = x_train.iloc[:,0:12].max(axis=1)
#x_train['min'] = x_train.iloc[:,0:12].min(axis=1)
#x_train['std'] = x_train.iloc[:,0:12].std(axis=1)

#x_test['max'] = x_test.iloc[:,0:12].max(axis=1)
#x_test['min'] = x_test.iloc[:,0:12].min(axis=1)
#x_test['std'] = x_test.iloc[:,0:12].std(axis=1)

#### Historical pressure

In [ ]:
x_test3 = hilbert_data1_jgrjd_20CRV3.data_pres(datetime.datetime(1905,1,1),datetime.datetime(2015,12,31))
x_test3.shape
#x_test3['doy'] = x_test3.index.dayofyear

In [ ]:
x_test3['doy'] = x_test3.index.month
#x_test3['max'] = x_test3.iloc[:,0:12].max(axis=1)
#x_test3['min'] = x_test3.iloc[:,0:12].min(axis=1)
#x_test3['std'] = x_test3.iloc[:,0:12].std(axis=1)
x_test3.shape

In [ ]:
plt.plot(x_test3.iloc[:,0],label = 'total _period')
plt.plot(x_train.iloc[:,0], label = 'train')
plt.plot(x_test.iloc[:,0], label = 'test')
plt.legend()
plt.ylabel('RMM2')

#### scale the data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc3 = MinMaxScaler()
sc5 = MinMaxScaler()

sc5.fit(x_test3[:])

test_x3 =  sc5.transform(x_test3[:])
train_x = sc5.transform(x_train[:])
test_x  = sc5.transform(x_test[:])


sc3.fit(y_train[:])

train_y = sc3.transform(y_train)
test_y  = sc3.transform(y_test)

train_x.max(),test_x.max(),test_x3.max(),train_y.max(),test_y.max()

In [ ]:
plt.hist(test_x[:,5],bins=35)

In [ ]:
#model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror'))
model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror'))

In [ ]:
model.estimator

#### Predict from XGBOOST for rmm2

In [ ]:
model.fit(train_x, train_y)

Y_prediction_rmm1 = model.predict(test_x)

acc_xgboost       = round(model.score(test_x, test_y) * 100, 2)

print("Accuracy of base Xgboost model is ,",acc_xgboost)

In [ ]:
predict1   = model.predict(train_x)
yy_train   = sc3.inverse_transform(predict1)
yy_train   = yy_train/yy_train.std()
train_corr = np.corrcoef(yy_train[:,0],train_y[:,0])[0,1]

print("training shape = ");print(predict1.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_train[:,0],train_y,'.')

ax[0].set_title('corr = %f'%train_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_train.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_train,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
predict2  = model.predict(test_x)
yy_test   = sc3.inverse_transform(predict2)
yy_test   = yy_test/yy_test.std()
test_corr = np.corrcoef(yy_test[:,0],test_y[:,0])[0,1]

print("test shape = ");print(predict2.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_test[:,0],test_y,'.')

ax[0].set_title('corr = %f'%test_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_test.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_test,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

## Support vector regressor

In [ ]:
from sklearn.svm import SVR# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.regressor = SVR(kernel='rbf')
regressor = SVR(kernel='rbf',C=5)

regressor.fit(train_x, train_y)

Y_prediction_rmm1 = regressor.predict(test_x)

acc_svr       = round(regressor.score(test_x, test_y) * 100, 2)

print("Accuracy of base SVR model is ,",(acc_svr/100)**0.5)

In [ ]:
predict1   = regressor.predict(train_x)
yy_train   = sc3.inverse_transform(predict1[:,None])
yy_train   = yy_train/yy_train.std()
train_corr = np.corrcoef(yy_train[:,0],train_y[:,0])[0,1]

print("training shape = ");print(predict1.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_train[:,0],train_y,'.')

ax[0].set_title('corr = %f'%train_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_train.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_train,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
predict2  = regressor.predict(test_x)
yy_test   = sc3.inverse_transform(predict2[:,None])
yy_test   = yy_test/yy_test.std()
test_corr = np.corrcoef(yy_test[:,0],test_y[:,0])[0,1]

print("test shape = ");print(predict2.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_test[:,0],test_y,'.')

ax[0].set_title('corr = %f'%test_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_test.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_test,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
predict_tot = regressor.predict(test_x3)
predict_tot = sc3.inverse_transform(predict_tot[:,None])
predict_tot = predict_tot/predict_tot.std()

## Neural Network

In [ ]:
from keras.optimizers import adadelta
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.optimizers import adamax
from keras.wrappers.scikit_learn import KerasRegressor
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=1000,  kernel_initializer='random_uniform',
                bias_initializer='zeros',input_dim=train_x.shape[1],activation='relu'))
    regressor.add(Dropout(0.1))
    regressor.add(Dense(units=1000, kernel_initializer='random_uniform',
                bias_initializer='zeros',activation='relu'))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=1000, kernel_initializer='random_uniform',
                bias_initializer='zeros',activation='relu'))
    regressor.add(Dropout(0.3))
    regressor.add(Dense(units=1, kernel_initializer='random_uniform',
                bias_initializer='zeros',activation='linear'))
    optimizer = adadelta(lr=0.1,epsilon=1e-8,rho=0.95)
    regressor.compile(optimizer=optimizer, loss='mae')
    return regressor

In [ ]:
regressor = KerasRegressor(build_fn=build_regressor, validation_data = (test_x,test_y),batch_size=30,epochs=50)

In [ ]:
results = regressor.fit(train_x,train_y,verbose=1)

In [ ]:
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])


In [ ]:
predict1   = regressor.predict(train_x)
yy_train   = sc3.inverse_transform(predict1.reshape(predict1.shape[0],1))
yy_train   = yy_train/yy_train.std()
train_corr = np.corrcoef(yy_train[:,0],train_y[:,0])[0,1]

print("training shape = ");print(predict1.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_train[:,0],train_y,'.')

ax[0].set_title('corr = %f'%train_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_train.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_train,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
predict2  = regressor.predict(test_x)
yy_test   = sc3.inverse_transform(predict2.reshape(predict2.shape[0],1))
yy_test   = yy_test/yy_test.std()
test_corr = np.corrcoef(yy_test[:,0],test_y[:,0])[0,1]

print("test shape = ");print(predict2.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_test[:,0],test_y,'.')

ax[0].set_title('corr = %f'%test_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_test.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_test,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
## LSTM

train_x_t = train_x[:,:,None]
test_x_t = test_x[:,:,None]

from keras.models import Sequential
from keras.layers import Dense,LSTM,Bidirectional
from keras.optimizers import Adam
import keras.backend as K


np.random.seed(7)

K.clear_session()
model = Sequential()

model.add(Bidirectional(LSTM(40, input_shape=(24,1))))
model.add(Dropout(0.2))
model.add(Dense(units=1000,  kernel_initializer='random_uniform',
                bias_initializer='zeros',input_dim=train_x.shape[1],activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1000, kernel_initializer='random_uniform',
            bias_initializer='zeros',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1000, kernel_initializer='random_uniform',
            bias_initializer='zeros',activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, kernel_initializer='random_uniform',
            bias_initializer='zeros',activation='linear'))
#model.add(Dense(1))
optimizer = adadelta(lr=0.1,epsilon=1e-8,rho=0.95)
#model.compile(loss='mae', optimizer=Adam(lr=0.001,decay=0.0001))
model.compile(loss='mae', optimizer=optimizer)
model.fit(train_x_t, train_y, validation_data=(test_x_t, test_y), epochs=100, verbose=1)

In [ ]:
predict2  = model.predict(test_x_t)
yy_test   = sc3.inverse_transform(predict2.reshape(predict2.shape[0],1))
yy_test   = yy_test/yy_test.std()
test_corr = np.corrcoef(yy_test[:,0],test_y[:,0])[0,1]

print("test shape = ");print(predict2.shape)

fig,ax = plt.subplots(1,2,figsize = (12,4))

ax[0].plot(yy_test[:,0],test_y,'.')

ax[0].set_title('corr = %f'%test_corr)

bins = np.linspace(-4.0,4.0,41)

ax[1].hist(y_test.rmm2,bins,density=True,alpha=0.5,label='observed')
ax[1].hist(yy_test,bins,density=True,alpha=0.5,label='modeled');
plt.legend()

In [ ]:
model.history.history.keys()

plt.plot(model.history.history['val_loss'])
plt.plot(model.history.history['loss'])